In [1]:

import os as os
import fnmatch as fnm
import numpy as np
import pandas as pd

fpkm_root = '/TL/deep/fhgfs/data/incoming/mirror/IKMB/human/GRCh37/41'
tracking_header = 'tracking_id class_code  nearest_ref_id  gene_id gene_short_name tss_id  locus   length  coverage    FPKM    FPKM_conf_lo    FPKM_conf_hi    FPKM_status'.split()
tracking_use_cols = 'gene_id gene_short_name locus FPKM'.split()

cache_dir = '/TL/deep/fhgfs/projects/pebert/collab/deepliver/cache'

genemodel = '/TL/deep/fhgfs/projects/pebert/thesis/refdata/genemodel/bed_format/gencode.v19.annotation.bed.gz'

def collect_files(topfolder):
    gene_quant = []
    for root, dirs, files in os.walk(topfolder):
        if files:
            fpkm = fnm.filter(files, '*LiHe*mRNA*_genes.fpkm_tracking')
            for f in fpkm:
                gene_quant.append(os.path.join(root, f))
    return gene_quant

def read_quant_file(fp):
    
    df = pd.read_csv(fp, delimiter='\t', header=0, usecols=tracking_use_cols)
    df.drop_duplicates(subset=['gene_id'], keep='first', inplace=True)
    mito_genes = df['locus'].str.startswith('MT')
    df = df.loc[~mito_genes, :]
    df.index = df['gene_id'].str.extract('(?P<ENSID>[A-Z0-9]+)', expand=False)
    df.drop(['gene_id'], inplace=True, axis=1)
    sample_name = os.path.basename(fp).split('.')[0].split('_', 1)[1].rsplit('_', 1)[0]
    proc_data = df.copy()
    rem = [c for c in proc_data.columns if c != 'FPKM']
    proc_data.drop(rem, axis=1, inplace=True)
    proc_data.columns = [sample_name]
    return df, proc_data, sample_name
    
def collect_exp_data():
    gene_quant = collect_files(fpkm_root)
    cache_file = os.path.join(cache_dir, 'raw_gene_quant.h5')
    cache_mode = 'w'
    exp_df = None
    for gf in gene_quant:
        raw_data, proc_data, sample = read_quant_file(gf)
        if exp_df is None:
            exp_df = proc_data
        else:
            exp_df = pd.concat([exp_df, proc_data], axis=1, ignore_index=False, join='outer')
        with pd.HDFStore(cache_file, cache_mode) as hdf:
            hdf.put(sample, raw_data, format='fixed')
        cache_mode = 'a'
    cache_exp = os.path.join(cache_dir, 'exp_gene_quant.h5')
    with pd.HDFStore(cache_exp, 'w') as hdf:
        hdf.put('fpkm', exp_df, format='fixed')
        hdf.flush()
    hdf.close()
    return exp_df

def identify_stable_genes(dataset):
    
    always_on = (dataset >= 5).all(axis=1)
    subset = dataset.loc[always_on, :]
    ranked = subset.rank(axis=0, pct=True)
    stable_genes = set()
    for lo in np.arange(0, 1, 0.1):
        hi = lo + 0.1
        lowly = (lo < ranked).all(axis=1)
        highly = (ranked <= hi).all(axis=1)
        joined = np.logical_and(lowly, highly)
        stable_genes = stable_genes.union(set(subset.loc[joined, :].index))
    return stable_genes


def dump_hk_genes(geneset, rawfile, strands, outputfile):
    
    with pd.HDFStore(rawfile, 'r') as hdf:
        some_data = list(hdf.keys())[0]
        raw = hdf[some_data]
    raw['chrom'] = raw['locus'].str.extract('(?P<CHROM>[0-9XY]+):', expand=False)
    raw['start'] = raw['locus'].str.extract(':(?P<START>[0-9]+)-', expand=False)
    raw['end'] = raw['locus'].str.extract('-(?P<END>[0-9]+)', expand=False)
    raw['name'] = raw.index
    raw.drop(['locus', 'FPKM'], inplace=True, axis=1)
    dump = raw.loc[geneset, :]
    add_strand = strands.loc[geneset, :]
    merged = pd.concat([dump, add_strand], axis=1, join='outer', ignore_index=False)
    merged.sort_values(by=['chrom', 'start', 'end'], axis=0, inplace=True)
    with open(outputfile, 'w') as out:
        _ = out.write('\t'.join(['#chrom', 'start', 'end', 'name', 'symbol', 'strand']) + '\n')
        for row in merged.itertuples():
            if row.strand == '-':
                line = '\t'.join([row.chrom, row.start, str(int(row.end) + 2000),
                                  row.name, row.gene_short_name, row.strand])
                
            else:
                line = '\t'.join([row.chrom, str(int(row.start) - 2000), row.end,
                                  row.name, row.gene_short_name, row.strand])
            _ = out.write(line + '\n')
    return

def read_genemodel(fp):
    df = pd.read_csv(fp, delimiter='\t', header=0,
                     usecols=['gene_id', 'strand'], skip_blank_lines=True)
    df['name'] = df['gene_id'].str.extract('(?P<ENSID>ENS[0-9A-Z]+)', expand=False)
    df.drop(['gene_id'], axis=1, inplace=True)
    df.drop_duplicates(subset=['name'], keep='first', inplace=True)
    df.index = df['name']
    df.drop(['name'], axis=1, inplace=True)
    return df

cached_expression = os.path.join(cache_dir, 'exp_gene_quant.h5')
if not os.path.isfile(cached_expression):
    df = collect_exp_data()

with pd.HDFStore(cached_expression, 'r') as expfile:
    expdat = expfile['fpkm']

hk_genes = identify_stable_genes(expdat)

gene_strands = read_genemodel(genemodel)

dump_hk_genes(hk_genes,
              os.path.join(cache_dir, 'raw_gene_quant.h5'),
              gene_strands,
              os.path.join(cache_dir, 'liver_hk_genes.bed'))
